# Web Scrape labelled images of building styles

# Architectural Index for Ontario - Archindont
* Archindont is a database of architectural information and citations to periodical articles and books about buildings in Toronto.
* list of building types: http://archindont.torontopubliclibrary.ca/Arch/search.do;jsessionid=jzKVTS6juZ65uhz4EA9tv9K7
http://archindont.torontopubliclibrary.ca/Arch/main.do

In [2]:
# Import libaries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import string
import re
import urllib
import os
import time
from datetime import datetime
import string

In [3]:
def load_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Error connecting: status code {response.status_code}")

In [4]:
def building_types_pages(type_url):
    '''
    Returns list of dictionaries where each dictionary is info about one building
    To Test: /Arch/buildingType.do;jsessionid=A2ZMTUjOxuCFc4yaAxah-Nws?type=Apartment+Houses&typeID=18'
    '''
    buildings=[]
    flags_dict = {
            'Address:':None,
            'Type of Building:': None,
            'Name of Building:': None,
            'Notes:':None,
            'Building completed': None,
            'Demolished': None,
           'Architects:': None
        }
    throw_flag = False
    flag = None

    
    type_url = f"{site_root}/{type_url}"
    html = load_page(type_url)
    soup = BeautifulSoup(html)
    for tag in soup.recursiveChildGenerator():
        if tag.name in ['th','td']:
            
            if tag.text == 'Address:' and flags_dict['Address:'] != None:
                # new building but skip the first building in the page
                buildings.append(flags_dict) # write out dict for prev building
                flags_dict = {
                    'Address:':None,
                    'Type of Building:': None,
                    'Name of Building:': None,
                    'Notes:':None,
                    'Building completed': None,
                    'Demolished': None,
                   'Architects:': None
                }
            if throw_flag:
               # print(f"{flag} {tag.text.strip()}")
                flags_dict[flag] = tag.text.strip()
                throw_flag = False
            if tag.text.strip() in flags_dict.keys():
               # print(tag.text)
                throw_flag = True
                flag = tag.text.strip()

    return buildings    

### Set up base variables

In [5]:
# search by Builidng Type
main_page = 'http://archindont.torontopubliclibrary.ca/Arch/search.do?searchType=Typ&initial='
#style_url="http://www.acotoronto.ca/search_buildingsDB_d2.php"
site_root = "http://archindont.torontopubliclibrary.ca"
debug=False
buildings_list=[]


In [23]:
for char in string.ascii_uppercase[25]:
    letter_url = f"{main_page}{char}"
    html = load_page(letter_url)
    soup = BeautifulSoup(html)
    bld_types = soup.find('form', {'name':'SearchResultsForm'}).findChildren('li')
    for style in bld_types:
        bld_type_url = style.find('a').get_attribute_list('href')[0]
        buildings_list.extend(building_types_pages(bld_type_url))
        time.sleep(5)
    bld_df = pd.DataFrame(buildings_list)
    bld_df.to_csv('../data/archidont_buildings_'+ str(char) + str(round(time.time(),0)) + '.csv')


In [60]:
bld_df = pd.DataFrame(buildings_list)

In [62]:
bld_df.head(20)

,Address:,Architects:,Building completed,Demolished,Name of Building:,Notes:,Type of Building:
0,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None
2,0 Airport Road,Bregman + Hamann Architects\nScott Associates ...,1991,None,Terminal 3,"Terminal 3, also known as the Trillium Termina...",Airport Terminals
3,0 Airport Road,Webb Zerafa Menkes Housden (firm),None,None,Lester B. Pearson Airport,"SEE ALSO: Terminal 1, Terminal 2 and Termincal...",Airport Terminals
4,0 Airport Road,Parkin Partnership (firm),1972,None,Terminal 2,"Will be replaced c2003 with a new, unified ter...",Airport Terminals
5,0 Airport Road,Parkin John B.,1964,None,Terminal 1,"Along with Terminal 2, will be replaced by new...",Airport Terminals
6,0 Airport Road,Adamson Associates (firm)\nSafdie Moshe\nSkidm...,None,None,None,New building to replace John C. Parkin's 1960'...,Airport Terminals
7,None,None,None,None,None,None,None
8,None,None,None,None,None,None,None
9,0 Exhibition Place,Burke Edmund\nChapman & Oxley (firm),None,None,Canadian National Exhibition,None,Amusement Parks


In [63]:
bld_df.to_csv('../data/archidont_buildings_'+ str(round(time.time(),0)) + '.csv')

In [ ]:
#